In [1]:
# Import necessary libraries
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import networkx as nx
import os,sys


notebook_path = os.path.abspath("")
sys.path.append(os.path.abspath(os.path.join(notebook_path, '..')))
from engine.core_rec import *



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/visheshyadav/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/visheshyadav/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/Users/visheshyadav/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start
    self.io_loop.start()

AttributeError: _ARRAY_API not found

In [2]:

# Import custom modules
from torch_geometric.datasets import Planetoid
from torch_geometric.data import Data

# Workaround for __file__ not being defined in Jupyter notebooks
notebook_path = os.path.abspath("")
sys.path.append(os.path.abspath(os.path.join(notebook_path, '..')))


# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

# Convert data to torch tensors
x = data.x
edge_index = data.edge_index

# Create a PyTorch Geometric Data object
data = Data(x=x, edge_index=edge_index)

# Add train_mask and test_mask
num_nodes = x.size(0)
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[:int(0.8 * num_nodes)] = True
test_mask = ~train_mask

data.train_mask = train_mask
data.test_mask = test_mask

# Add labels (y) if not already present
num_classes = dataset.num_classes
data.y = data.y if data.y is not None else torch.randint(0, num_classes, (num_nodes,), dtype=torch.long)


In [3]:
adj_matrix = nx.to_numpy_array(nx.from_edgelist(edge_index.t().tolist()))
train_dataset = GraphDataset(adj_matrix[train_mask])
test_dataset = GraphDataset(adj_matrix[test_mask])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:

# Initialize the model
num_layers = 1
d_model = 128
num_heads = 8
d_feedforward = 512
input_dim = 2708 # Ensure this matches the number of features in the dataset
model = GraphTransformerV2(num_layers, d_model, num_heads, d_feedforward, input_dim)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=100)
# Evaluate the model
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, targets in test_loader:
        # print(f"Input shape: {inputs.shape}")  # Add this line to check the input shape
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        # print(f"Preds shape: {preds.shape}, Targets shape: {targets.shape}")  # Debugging shapes
        targets = torch.argmax(targets, dim=1)  # Convert one-hot encoded targets to class indices
        all_preds.extend(preds.cpu().numpy().flatten())  # Ensure preds are flattened
        all_labels.extend(targets.cpu().numpy().flatten())  # Ensure targets are flattened
        # print(f"Current length of all_preds: {len(all_preds)}, all_labels: {len(all_labels)}")  # Debugging lengths

# Convert to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Debugging: Print final lengths of all_preds and all_labels
# print(f"Final length of all_preds: {len(all_preds)}")
# print(f"Final length of all_labels: {len(all_labels)}")

# Ensure both arrays are 1-dimensional
all_preds = all_preds.flatten()
all_labels = all_labels.flatten()

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {accuracy:.4f}")

AttributeError: 'Tensor' object has no attribute 'x'

In [45]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
import matplotlib.pyplot as plt

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Define a simple GAT model
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads=8):
        super(GAT, self).__init__()
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=0.6)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, heads=1, concat=False, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Define the GraphTransformer model (assuming the class is already defined)
# from your previous implementation
# class GraphTransformer(torch.nn.Module):
#     # Your GraphTransformer implementation
#     pass

# Define a simple MLP model
class MLP(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x = data.x
        x = self.fc1(x).relu()
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Function to train a model
def train_model123(model, data, epochs=200, lr=0.01, weight_decay=5e-4):
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    model.train()
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

# Function to evaluate a model
def evaluate_model(model, data):
    model.eval()
    _, pred = model(data).max(dim=1)
    correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    acc = correct / data.test_mask.sum().item()
    return acc

# Load data
data = dataset[0]

# Define models
input_dim = dataset.num_node_features
hidden_dim = 16
output_dim = dataset.num_classes
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
import sys,os
notebook_path = os.path.abspath("")
sys.path.append(os.path.abspath(os.path.join(notebook_path, '..')))
from engine.core_rec import *

models = {
    'GCN': GCN(input_dim, hidden_dim, output_dim),
    'GAT': GAT(input_dim, hidden_dim, output_dim),
    'MLP': MLP(input_dim, hidden_dim, output_dim),
    # Add your GraphTransformer here with the appropriate parameters
    'GraphTransformer': GraphTransformer(num_layers=2, d_model=hidden_dim, num_heads=2, d_feedforward=hidden_dim, input_dim=input_dim)
}
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Train and evaluate models
accuracies = {}
for name, model in models.items():
    print(f"Training {name}...")
    if name != 'GraphTransformer':
        train_model1(model, data, epochs=10, lr=0.01, weight_decay=5e-4)  # Corrected function call
    else:
        train_model123(model, data, criterion, optimizer)  # Corrected function call
    acc = evaluate_model(model, data)
    accuracies[name] = acc
    print(f"{name} Accuracy: {acc:.4f}")

# Plot accuracies
plt.figure(figsize=(10, 6))
plt.bar(accuracies.keys(), accuracies.values(), color='skyblue')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Model Accuracies on Cora Dataset')
plt.show()

Training GCN...
GCN Accuracy: 0.7400
Training GAT...
GAT Accuracy: 0.7980
Training MLP...
MLP Accuracy: 0.4050
Training GraphTransformer...


TypeError: '<=' not supported between instances of 'float' and 'Adam'

In [35]:
import core_rec as cr

In [40]:
def train_model1(model, data, epochs=200, lr=0.01, weight_decay=5e-4):
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    model.train()
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
